In [2]:
import pickle
from shapely import LineString

import geopandas as gpd
import osmnx as ox
from scipy.spatial import cKDTree
import networkx as nx

from my_paths import *
import Graph_functions

In [3]:
PATH_STRADE_CICLABILI_PICKLE_RAW = "../Data/Raw/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.pickle"
PATH_STRADE_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/strade_ciclabili.geojson"

PATH_CICLABILI_PICKLE_STAGING = "../Data/Staging/Info_bici/ciclabile.pickle"
PATH_CICLABILI_GEOJSON_STAGING = "../Data/Staging/Info_bici/ciclabile.geojson"
PATH_GEOJSON_STRADE_E_CICLABILI_STAGING = "../Data/Staging/Analisi/strade_e_ciclabili.geojson"
PATH_GEOJSON_COMPLETO_STAGING = "../Data/Staging/Analisi/completo.geojson"

PATH_GEOJSON_PERCORSI_STAGING = "../Data/Staging/Analisi/percorsi.geojson"

PATH_TEST_STAGING = "../Data/Staging/Analisi/"

## Creazione Strade ciclabili Graph (compreso ingestion da openstreet maps)

Ingestion da openstreet maps e salviamo in Raw (pickle)

In [52]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

Carichiamo il file da Raw e modifichiamolo

In [17]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)

Aggiungiamo un peso in base alla lunghezza della strada

In [18]:
for u, v, k, data in G_strade.edges(data=True, keys=True):
    # Scelgo gli attributi da mantenere
    new_data = {}
    new_data['length'] = data['length']
    new_data['name'] = data.get('name', None)
    new_data['weight'] = data['length'] * 1 # Per ora
    new_data["tipo"] = "Strade_ciclabili"
    new_data["artificiale"] = False
    new_data["geometry"] = data.get("geometry", LineString([(G_strade.nodes[u]["x"], G_strade.nodes[u]["y"]),
                                        (G_strade.nodes[v]["x"], G_strade.nodes[v]["y"])]))
    data.clear()
    data.update(new_data)

Aggiungiamo archi di congiunzione artificiali per connettere strade molto vicine tra di loro

In [19]:
G_strade = G_strade.to_undirected()
G_strade = Graph_functions.add_edge_near_nodes(G_strade,
                                               weight_moltiplicator=1,  # Per ora
                                               tipo = "Strade_ciclabili", artificiale = True)

Salva GeoJson & Pickle in staging

In [20]:
_, gdf_edges = ox.graph_to_gdfs(G_strade)

# Andrebbe pulito il geojson ma per ora lo usiamo solo per visualizzazione su kepler
gdf_edges.to_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING, driver="GeoJSON")

with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)

## Creazione Ciclabili Graph

Modifico ad hoc il raw delle ciclabili. Se decideremo di utilizzare questo metodo dei graph, modificheremo l'ETL delle ciclabili in questo modo e aggiungeremo la conversione in graph e il salvataggio in pickle

In [21]:
# Carico Raw
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo Multilinestring in Linestring
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])

# Altre operazioni di pulizia. Da considerare di rimuover il group_by dall'ETL delle piste
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

G_ciclabili = Graph_functions.geojson_to_graph(gdf_ciclabili_staging, weight_moltiplicator=0, tipo = "Ciclabile", artificiale = False)
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

Anche qui aggiungiamo connessioni artificiali tra nodi vicini non connessi

In [22]:
# Carichiamo MultiDiGraph da file pickle
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Aggiungiamo archi artificiali
G_ciclabili = G_ciclabili.to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili, weight_moltiplicator = 0, tipo = "Ciclabile", artificiale = True)

# Salviamo pickle e geojson in staging
_, gdf_edges = ox.graph_to_gdfs(G_ciclabili)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

## Proviamo a creare connessioni con networkx

Carichiamo da file i Graph (dai file pickle)

In [57]:
# Carichiamo Graph Ciclabili
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Carichiamo Graph Strade
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_strade = pickle.load(f)

Uniamo in un unico graph e aggiungiamo le connessioni tra i due con lo stesso meccanismo usato prima

In [58]:
for n in G_ciclabili.nodes:
    G_ciclabili.nodes[n]["origin"] = "strada"
for n in G_strade.nodes:
    G_strade.nodes[n]["origin"] = "pista"
G_compose = nx.compose(G_strade, G_ciclabili)
G_compose_merged = Graph_functions.add_edge_near_nodes(G_compose,
                                                       weight_moltiplicator = 0,
                                                       tipo = "Join_ciclabile_strade",
                                                       artificiale=True)

In [59]:
gdf_compose = ox.graph_to_gdfs(G_compose_merged, edges=True, nodes=False)
gdf_compose.to_file(PATH_GEOJSON_STRADE_E_CICLABILI_STAGING, driver="GeoJSON")

In [60]:
# Carichiamo Parchi GeoDataFrame
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)

G_compose_merged_parchi = Graph_functions.connect_poi_nodes_to_graph(G_compose_merged, gdf_parchi,
                                                                     tipo = "parchi",
                                                                     poi = True,
                                                                     artificial=True)

In [61]:
gdf_compose = ox.graph_to_gdfs(G_compose_merged_parchi, edges=True, nodes=False)
gdf_compose.to_file(PATH_GEOJSON_COMPLETO_STAGING, driver="GeoJSON")

## Test 1: nx.shortest_path() di tutte le combinazioni di P.O.I e poi nx.minimum_spanning_tree()

In [54]:
from itertools import combinations
from tqdm import tqdm 

poi_nodes = [n for n, d in G_compose_merged_parchi.nodes(data=True) if d.get("tipo") == "parchi"]

poi_pairs = list(combinations(poi_nodes, 2))

In [17]:
graph_complete = nx.Graph()
    
pbar = tqdm(total=len(list(poi_pairs)), desc="Estrazione tweet")
for u, v in poi_pairs:
    try:
        path = nx.shortest_path(G_compose_merged_parchi, source=u, target=v, weight='weight')
        length = sum(G_compose_merged_parchi[u][v][0]['weight'] for u,v in zip(path[:-1], path[1:]))
        graph_complete.add_edge(u, v, weight=length, path=path)
    except nx.NetworkXNoPath:
        continue
    pbar.update(1)

mst = nx.minimum_spanning_tree(graph_complete, weight='weight')

In [50]:
from shapely.geometry import LineString
import geopandas as gpd

edge_geoms = []
for u, v, data in mst.edges(data=True):
    path = data.get("path")
    if not path:
        continue
    coords = [(G_compose_merged_parchi.nodes[n]['x'], G_compose_merged_parchi.nodes[n]['y']) for n in path]
    geom = LineString(coords)
    edge_geoms.append({
        "u": u,
        "v": v,
        "geometry": geom,
        "length": data["weight"]
    })

gdf_paths = gpd.GeoDataFrame(edge_geoms, geometry="geometry", crs="EPSG:4326")

In [53]:
gdf_paths.to_file(PATH_GEOJSON_PERCORSI_STAGING, driver="GeoJSON")

## Test 2: Algoritmo di "steiner_tree"

In [ ]:
components = list(nx.connected_components(G_compose_merged_parchi))
largest_cc = max(components, key=len)

G_sub = G_compose_merged_parchi.subgraph(largest_cc).copy()
poi_sub = [n for n in poi_nodes if n in G_sub.nodes]

for u, v, data in G_sub.edges(data=True):
    if 'weight' not in data or not isinstance(data['weight'], (int, float)):
        data['weight'] = 1  # o un peso di default

from networkx.algorithms.approximation import steiner_tree

T = steiner_tree(G_sub, terminal_nodes=poi_sub, weight='weight')

Nodi POI mancanti nel grafo semplice: []
Nodi POI fuori dalla componente principale: []


In [73]:
gdf_steiner = ox.graph_to_gdfs(T, edges=True, nodes=False)
gdf_compose.to_file(PATH_GEOJSON_COMPLETO_STAGING, driver="GeoJSON")